In [5]:
# import tensorflow as tf

# gpus = tf.config.experimental.list_physical_devices('GPU')

# if gpus:

#     try:

#         for gpu in gpus:

#             tf.config.experimental.set_memory_growth(gpu, True)

#     except RuntimeError as e:

#         print(e)

# # 실행가능한 gpu 목록
# tf.config.list_physical_devices('GPU')
# #실행가능한 cpu, gpu 목록
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [1]:
import numpy as np
import pandas as pd
from data_split import data_split

import sys
sys.path.insert(0, '../')
from config.global_params import variable_info


# 인풋 준비
nwp_file = "../DAIO/nwp_data_47105"
obs_file = "../DAIO/obs_data_47105"
nwp_data = np.load(nwp_file)
obs_data = np.load(obs_file)
print("="*50, "load data shape")
print(nwp_data.shape)
print(obs_data.shape)


# train([21.01,04, 22.01,04]) / test([23.01,04]) 분할  
class_split = data_split(nwp_data, obs_data)
train_nwp, test_nwp, train_obs, test_obs = class_split.get_split_data()
print("="*50, "split data shape")
print(train_nwp.shape)
print(train_obs.shape)
print(test_nwp.shape)
print(test_obs.shape)



# 결측제거
missing_nwp_train = set(np.where(np.isnan(train_nwp))[0])
missing_obs_train = set(np.where(np.isnan(train_obs))[0])
missing_all_train = list(missing_nwp_train | missing_obs_train)
print("결측 합계: ", len(missing_all_train))
dm_nwp_train = np.delete(train_nwp, missing_all_train, 0)
dm_obs_train = np.delete(train_obs, missing_all_train, 0)
print("shape of after drop")
print(dm_nwp_train.shape)
print(dm_obs_train.shape)

missing_nwp_test = set(np.where(np.isnan(test_nwp))[0])
missing_obs_test = set(np.where(np.isnan(test_obs))[0])
missing_all_test = list(missing_nwp_test | missing_obs_test)
print("결측 합계: ", len(missing_all_test))
dm_nwp_test = np.delete(test_nwp, missing_all_test, 0)
dm_obs_test = np.delete(test_obs, missing_all_test, 0)
print("shape of after drop")
print(dm_nwp_test.shape)
print(dm_obs_test.shape)


# 변수선택
sel_var = ['NDNSW_surface', 'UGRD_10m', 'VGRD_10m', 'RH_1_5ma', 'MAXGUST_0m', 'PRMSL_meansealevel']
var_list_dict = list(variable_info.keys())
var_index = [ var_list_dict.index(i) for i in sel_var ]
#print(var_list_dict)
#print(var_index)
sel_dm_nwp_train = dm_nwp_train[:,:,var_index]
sel_dm_nwp_test = dm_nwp_test[:,:,var_index]
print("="*50, "drop data shape")
print(sel_dm_nwp_train.shape)
print(dm_obs_train.shape)
print(sel_dm_nwp_test.shape)
print(dm_obs_test.shape)

================================================== load data shape
(868, 49, 20)
(868, 49, 2)
================================================== split data shape
(109, 49, 20)
(109, 49, 2)
(61, 49, 20)
(61, 49, 2)
결측 합계:  1
shape of after drop
(108, 49, 20)
(108, 49, 2)
결측 합계:  5
shape of after drop
(56, 49, 20)
(56, 49, 2)
================================================== drop data shape
(108, 49, 6)
(108, 49, 2)
(56, 49, 6)
(56, 49, 2)


In [2]:
# .. 스케일링 및 데이터 분할
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#-------------------------------------------------------------------------
# .. Normalize

output_size = 2

# .. initialaize
tr_b, tr_s, tr_f = sel_dm_nwp_train.shape[0], sel_dm_nwp_train.shape[1], sel_dm_nwp_train.shape[2]      
ts_b, ts_s, ts_f = sel_dm_nwp_test.shape[0], sel_dm_nwp_test.shape[1], sel_dm_nwp_test.shape[2]      

# .. get restorator with obs range
nwp_scaler = MinMaxScaler()   # copy default true
obs_scaler = MinMaxScaler()
nwp_scaler.fit(sel_dm_nwp_train.view().reshape(tr_b*tr_s, tr_f))
obs_scaler.fit(dm_obs_train.view().reshape(tr_b*tr_s, output_size))

# .. feature normalize   ( train seq, feature = test seq, feature )
nor_dm_nwp_train = nwp_scaler.transform(sel_dm_nwp_train.reshape(tr_b*tr_s, tr_f))
nor_dm_nwp_train = nor_dm_nwp_train.reshape(tr_b,tr_s,tr_f)
nor_dm_obs_train = obs_scaler.transform(dm_obs_train.reshape(tr_b*tr_s, output_size))
nor_dm_obs_train = nor_dm_obs_train.reshape(tr_b,tr_s, output_size)

nor_dm_nwp_test = nwp_scaler.transform(sel_dm_nwp_test.reshape(ts_b*ts_s, ts_f))
nor_dm_nwp_test = nor_dm_nwp_test.reshape(ts_b,ts_s,ts_f)
nor_dm_obs_test = obs_scaler.transform(dm_obs_test.reshape(ts_b*ts_s, output_size))
nor_dm_obs_test = nor_dm_obs_test.reshape(ts_b,ts_s, output_size)

nor_dm_nwp_train = nor_dm_nwp_train[:,1::,:]
nor_dm_obs_train = nor_dm_obs_train[:,1::,:]

nor_dm_nwp_test = nor_dm_nwp_test[:,1::,:]
nor_dm_obs_test = nor_dm_obs_test[:,1::,:]

print ('---------- Final training data shape')
print(type(nor_dm_nwp_train))
print ('tran nwp : ', nor_dm_nwp_train.shape)
print ('tran obs : ', nor_dm_obs_train.shape)
print ('test nwp : ', nor_dm_nwp_test.shape)
print ('test obs : ', nor_dm_obs_test.shape)



---------- Final training data shape
<class 'numpy.ndarray'>
tran nwp :  (108, 48, 6)
tran obs :  (108, 48, 2)
test nwp :  (56, 48, 6)
test obs :  (56, 48, 2)


In [10]:
import os
import joblib
from time import time

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, TimeDistributed
from tensorflow.keras import Input, Model, callbacks
from tensorflow.keras.utils import plot_model as plm
from tensorflow.keras.activations import swish
from keras.callbacks import CSVLogger

from tensorflow.compat.v1.keras.backend import set_session
from scikeras.wrappers import KerasRegressor
from tcn import TCN, tcn_full_summary

#from skopt import BayesSearchCV
#from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint



#.. local
from sklearn.metrics import make_scorer, r2_score
from kmk_make_scorer import r2_3dim, mse_3dim, mae_3dim

#-------------------------------------------------------------------------
# .. Set configure

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config))


#-------------------------------------------------------------------------
# .. Data set

element = 'ALLV'
name_list = "./SHEL/namelist.input"

hp_lr = 0.009
hp_pd = 'same'
hp_ns = 1
hp_dl = [1,2,4,8,16,32,48]
hp_ldl = hp_dl[-1] # last dilation factor to make name of save model
hp_bn = True
hp_nf = 80
hp_dr = 0.07
hp_ks = 6

input_size = 6
output_size = 2
num_fct = 48
batch_size = 8
n_iter_search = 20
num_epoch = 600
dev_stn_id = 47105
tran_data_per = "2101_2104_2201_2204"

exp_name = "CNTL"
csv_outdir = './DAOU/LOSS/' + exp_name + '/'
model_outdir = './DAOU/MODL/' + exp_name + '/'
scalr_outdir = './DAOU/SCAL/' + exp_name + '/'
gifd_outdir = './GIFD/' + exp_name + '/'
log_outdir = './DAOU/LOGF/' + exp_name + '/'


if os.path.exists(csv_outdir) != True: os.makedirs(csv_outdir)
if os.path.exists(model_outdir) != True: os.makedirs(model_outdir)
if os.path.exists(scalr_outdir) != True: os.makedirs(scalr_outdir)
if os.path.exists(gifd_outdir) != True: os.makedirs(gifd_outdir)
if os.path.exists(log_outdir) != True: os.makedirs(log_outdir)


In [19]:
#=========================================================================
# .. Model configuration
num_cv = 5


#-------------------------------------------------------------------------
# .. Set batch for cross-validation

print ('input_size: ', input_size)
print ('batch_size: ', batch_size)
print ('time_lenght: ', num_fct)


#-------------------------------------------------------------------------
# .. Set Model


# .. Define model
def create_model(dropout_rate=0.15, nb_filters=7, kernel_size=3): 

          print ('================== Model called ========================')
          print ('input_size: ', input_size)
          print ('batch_size: ', batch_size)
          print ('time_lenght: ', num_fct)
          print ('nb_filters: ', nb_filters)
          print ('kernel_size: ', kernel_size)
          print ('dropout_rate: ', dropout_rate)
          print ('dilations: ', hp_dl)
          dropout_rate = np.round(dropout_rate,2)
          print ('dropout_rate: ', dropout_rate)
          
          ## .. clear keras model
          K.clear_session()

          # .. create model
          #i = Input( batch_shape=(batch_size, num_fct, input_size) )
          i = Input( batch_shape=(None, num_fct, input_size) )
          o = TCN(return_sequences=True,
                  activation=swish,
                  nb_filters=nb_filters,
                  padding=hp_pd,
                  use_batch_norm = hp_bn,
                  nb_stacks=hp_ns,
                  dropout_rate=dropout_rate,
                  kernel_size=kernel_size,
                  use_skip_connections=True,
                  dilations=hp_dl
                  )(i)
          o = TimeDistributed(Dense(output_size, activation='linear'))(o)

          # .. compile
          adam = optimizers.Adam(lr=hp_lr)

          m= Model(inputs=[i], outputs=[o])
          m.compile(optimizer=adam, loss='mse')

          m.summary()

          return m




input_size:  6
batch_size:  8
time_lenght:  48


In [22]:
#model = create_model()
#model.count_params

# .. Wrapping create_model for searching hyper-parameter
model = KerasRegressor(build_fn=create_model,
                       verbose=1,
                       epochs=num_epoch,
                       batch_size=batch_size,
                       shuffle=True)
model.get_params()

{'model': None,
 'build_fn': <function __main__.create_model(dropout_rate=0.15, nb_filters=7, kernel_size=3)>,
 'warm_start': False,
 'random_state': None,
 'optimizer': 'rmsprop',
 'loss': None,
 'metrics': None,
 'batch_size': 8,
 'validation_batch_size': None,
 'verbose': 1,
 'callbacks': None,
 'validation_split': 0.0,
 'shuffle': True,
 'run_eagerly': False,
 'epochs': 600}

In [23]:


#-------------------------------------------------------------------------
# .. Use bayes_opt

# .. Exp para set
#param_dist = { 'padding': Categorical(['causal','same']),
#               'nb_stacks': Integer(1,5),
#               'nb_filters': Categorical([7,20,30]),
#               'kernel_size': Integer(2,24) }
#param_dist = { 'dropout_rate': Real(0.01, 0.2),
#               'nb_filters': Integer(50,100),
#               'kernel_size': Integer(3,12) }
##param_dist = { 'dropout_rate': Categorical([0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.15]),
#               'nb_filters': Integer(50,100),
#               'kernel_size': Integer(3,12) }
param_dist = { 'nb_filters': randint(10, 100)}

#               'kernel_size': [3,6,9,12] }

set_eval_score = { 'MAE': make_scorer(mae_3dim),
                   'MSE': make_scorer(mse_3dim),
                   'R2': make_scorer(r2_3dim) }

print ( param_dist )

# optimizer =  BayesSearchCV( estimator=model,
#                             search_spaces=param_dist,
#                             scoring=make_scorer(r2_3dim),
#                             refit=False,
#                             cv=num_cv,
#                             n_iter=n_iter_search,
#                             return_train_score=True,
#                             verbose=1,
#                             n_jobs=1,
#                             random_state=1 )

optimizer =  RandomizedSearchCV( estimator=model,
                            param_distributions=param_dist,
                            scoring=make_scorer(r2_3dim),
                            refit=False,
                            cv=num_cv,
                            n_iter=n_iter_search,
                            return_train_score=True,
                            verbose=1,
                            n_jobs=-1,
                            random_state=1 )

start = time()
print(nor_dm_nwp_train.shape, nor_dm_obs_train.shape)
optimizer.fit(nor_dm_nwp_train, nor_dm_obs_train)

print(type(optimizer.cv_results_))
print(optimizer.cv_results_)

# .. Report
def report(result, n_top=n_iter_search):
    for i in range(n_top):
        candidates = [ result['rank_test_score'][i] ]
        for candidate in candidates:
            print("Rank: %0d, R2: %.3f with %r" %
                  ( i, result['mean_test_score'][candidate-1],
                       result['params'][candidate-1] ) )


print("BayesSearchCV took %.2f seconds for %d candidates"
      " parameter settings. " % ((time() - start), n_iter_search))

print( "Best: %f using %s" % ( optimizer.best_score_,
                               optimizer.best_params_ ) )

report(optimizer.cv_results_)


#=========================================================================
# .. Second refit to evaluate best model use test period


#-------------------------------------------------------------------------
# .. Set model label

# .. best model configuration for whole train set
params = optimizer.best_params_
print(type(params))
print(params)

{'nb_filters': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000204FF452250>}
(108, 48, 6) (108, 48, 2)
Fitting 5 folds for each of 20 candidates, totalling 100 fits


ValueError: Invalid parameter nb_filters for estimator KerasRegressor.
This issue can likely be resolved by setting this parameter in the KerasRegressor constructor:
`KerasRegressor(nb_filters=47)`
Check the list of available parameters with `estimator.get_params().keys()`

In [8]:
np.arange(3,15,3)

array([ 3,  6,  9, 12])